In [ ]:
CREATE OR REPLACE TEMPORARY VIEW orders (
    order_id INT,
    order_date STRING,
    order_customer_id INT,
    order_status STRING
) USING CSV
OPTIONS (
    path='dbfs:/public/retail_db/orders',
    sep=','
)

In [ ]:
CREATE OR REPLACE TEMPORARY VIEW order_items (
    order_item_id INT,
    order_item_order_id INT,
    order_item_product_id INT,
    order_item_quantity INT,
    order_item_subtotal FLOAT,
    order_item_product_price FLOAT
) USING CSV
OPTIONS (
    path='dbfs:/public/retail_db/order_items',
    sep=','
)

In [ ]:

INSERT INTO DIRECTORY 'dbfs:/public/retail_db/daily_product_revenue'
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal)) AS revenue
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi. order_item_order_id
WHERE order_status IN ('COMPLETE', 'CLOSED')
GROUP BY 1, 2

In [ ]:
%fs ls dbfs:/public/retail_db/daily_product_revenue

In [ ]:
SELECT * FROM PARQUET.`dbfs:/public/retail_db/daily_product_revenue`
ORDER BY order_date, revenue DESC

In [ ]:
SELECT * FROM PARQUET.`dbfs:/public/retail_db/daily_product_revenue`
ORDER BY 1, 3 DESC